# Input

In [ ]:
INVOICE_DIR = '/work/data/OpenIRIS/LMU-20200427' #@param {type:"raw"}
INVOICE_FILE = 'Invoice24.xlsx' #@param {type:"raw"}
FIXES_FILE = 'fixed_Invoice24__price_type_missing.xlsx' #@param {type:"raw"}

from datetime import datetime
import pandas as pd
from pathlib import Path

INVOICE_DIR = Path(INVOICE_DIR)
if not INVOICE_DIR.is_dir():
    raise ValueError('Please check INVOICE_DIR.')
INVOICE_FILE = INVOICE_DIR / INVOICE_FILE
if not INVOICE_FILE.exists():
    raise ValueError('Please check INVOICE_FILE.')
FIXES_FILE = INVOICE_DIR / FIXES_FILE
if not FIXES_FILE.exists():
    raise ValueError('Please check FIXES_FILE.')

# read first two rows of input
header = pd.read_excel(INVOICE_FILE, nrows=1)

# check if the first two rows are the invoice summary from IRIS
if 'Created by' in header.columns:
    print('input file has IRIS summary, skip it')
    df = pd.read_excel(INVOICE_FILE, skiprows=[0,1])
else:
    print('no IRIS summary, read entire file')
    df = pd.read_excel(INVOICE_FILE)

# backup original
timestamp = str(datetime.now().strftime("%Y%m%d-%H%M%S"))
print(timestamp)
df.to_excel(INVOICE_DIR / (INVOICE_FILE.stem + '__' + timestamp + '.xlsx'), index=False)

df2 = pd.read_excel(FIXES_FILE)

# use the original index in column 'Unnamed: 0'
df2.set_index('Unnamed: 0', inplace=True)
df2.head(3)


# Modifications only (no added rows)

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.update.html?highlight=update#pandas.DataFrame.update
df3 = df.copy()
df3.update(df2)

# save the modifications
OUTPUT = INVOICE_DIR / (INVOICE_FILE.stem + "__update__" + FIXES_FILE.stem + INVOICE_FILE.suffix)
df3.to_excel(OUTPUT,index=False)
print(OUTPUT)

# overwrite original
df3.to_excel(INVOICE_FILE,index=False)
